In [16]:
from solid import hull, cylinder, translate
from solid.objects import cube, minkowski, rotate
from viewscad import Renderer

r = Renderer()
r.width = 700
r.height = 700

render = r.render


def ring(r1=10, r2=None, h=2, w=2, dx=0, dy=0, dz=0):
    r2 = r2 or r1
    outer = cylinder(r1=r1, r2=r2, h=h, segments=40)
    inner = cylinder(r1=r1-w, r2=r2-w, h=h, segments=40)
    return translate((dx, dy, dz))(outer-inner)


def slot(*args, angle=30, **kwargs):
    half = ring(*args, **kwargs) - translate((-50, -100, 0))(cube(100))

    if angle == 180:
        return half
    elif angle < 180:
        return half * rotate((0, 0, 180+angle))(half)
    else:
        return half + rotate((0, 0, angle))(half)


bottom = ring(r1=10, h=2, w=2)\
    + ring(r1=8, h=4, w=2)\
    + ring(r1=10, r2=8, h=2, w=2, dz=2)\
    + ring(r1=8, h=2, w=2, dz=4) \
    + ring(r1=8, r2=12, h=6, w=2, dz=6)\
    + ring(r1=12, dz=12)

middle = ring(r1=12, dz=14, h=20)
slots = sum([rotate((0, 0, r))(slot(r1=10.5, w=2.5, h=4, angle=40))
            for r in [0, 120, 240]])
stopper = sum([rotate((0, 0, r))(translate((6, -2, 0))(cube((4, 2, 10)))) for r in [0, 120, 240]]) \
    * (ring(r1=10, h=9) + ring(r1=8, r2=12, h=6, w=2, dz=6))

hook = slot(r1=10, h=2, dz=36)\
    + slot(r1=12, r2=10, h=2, dz=34)\
    + slot(r1=10, r2=12, h=2, dz=38)\
    + ring(r1=12, h=6, dz=34)\
    + (ring(r1=12, h=2, dz=40)
       * ring(r1=12, r2=14, h=2, dz=40))

top = sum([rotate((0, 0, r))(hook) for r in [0, 120, 240]])

scene = bottom + middle - slots + top + stopper
render(scene, outfile='test_pipe.stl')
